In [ ]:
!pip install locust

In [ ]:
!which locust

In [ ]:
import boto3
from botocore.config import Config
import pandas as pd
import itertools
import datetime

region = 'us-east-1'
content_type = 'text/csv'
endpoint_name='sagemaker-decision-trees-2021-03-17-03-32-20-604'

boto3config = Config(
    retries={
        'max_attempts': 100,
        'mode': 'standard'
    }
)

payload='5.0,3.5,1.3,0.3\n'

sagemaker_client = boto3.client('sagemaker-runtime',
                                     config=boto3config,
                                     region_name=region)

response = sagemaker_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=payload,
    ContentType=content_type
)
response_body = response["Body"].read()

response_body

In [ ]:
input_ids = []
attention_mask = []
import json

# open file in write mode
with open(r'../../temp-bloom/input_ids.txt', 'r') as fp:
    for line in fp:
        # remove linebreak from a current name
        # linebreak is the last character of each line
        x = line[:-1]

        # add current item to the list
        input_ids.append(int(x))
    print(f"Done input_ids size={len(input_ids)}::")
    
# open file in write mode
with open(r'../../temp-bloom/attention_mask.txt', 'r') as fp:
    for line in fp:
        # remove linebreak from a current name
        # linebreak is the last character of each line
        x = line[:-1]

        # add current item to the list
        attention_mask.append(int(x))
    print(f"Done attention_mask size={len(attention_mask)}::")


In [ ]:
input_ids

In [ ]:
max_seq_length=512
payload = {
    "inputs": [
        {"name": "INPUT__0", "shape": [1, max_seq_length], "datatype": "INT32", "data": input_ids},
        {"name": "INPUT__1", "shape": [1, max_seq_length], "datatype": "INT32", "data": attention_mask},
    ]
}

with open('./payload_json.json', 'w') as fp:
    json.dump(payload, fp)

In [ ]:
payload_read = None
with open('./payload_json.json', 'r') as fp:
    payload_read = json.load(fp)
payload_read.keys()

In [ ]:
%%time
import json
import boto3
from botocore.config import Config
import pandas as pd
import itertools
import datetime

region = 'us-east-1'
content_type = 'text/csv'
endpoint_name_p5='p5-bert-uc--2022-09-08-03-02-53-774'

boto3config = Config(
    retries={
        'max_attempts': 100,
        'mode': 'standard'
    }
)


sagemaker_client = boto3.client('sagemaker-runtime',
                                     config=boto3config,
                                     region_name=region)

max_seq_length=512
print(f"Leverage the stored:ids: max_seq_length={max_seq_length}:: create above when creating the model : INPUTSid's and attention masks read earlier")
#input_ids, attention_mask = tokenize_text(text_triton, enc, max_length=max_seq_length)

payload = {
    "inputs": [
        {"name": "INPUT__0", "shape": [1, max_seq_length], "datatype": "INT32", "data": input_ids},
        {"name": "INPUT__1", "shape": [1, max_seq_length], "datatype": "INT32", "data": attention_mask},
    ]
}

response = sagemaker_client.invoke_endpoint(
    EndpointName=endpoint_name_p5, ContentType="application/octet-stream", Body=json.dumps(payload), 
)

output_dict = json.loads(response["Body"].read().decode("utf8"))

# -- output_dict['outputs'][0]['data']  -- has 0 and 1 as 2 indexes in list 
print(output_dict.keys())


response = sagemaker_client.invoke_endpoint(
    EndpointName=endpoint_name_p5, ContentType="text/json", Body=json.dumps(payload)
)
output_dict = json.loads(response["Body"].read().decode("utf8"))

# -- output_dict['outputs'][0]['data']  -- has 0 and 1 as 2 indexes in list 
print(output_dict.keys())


In [ ]:
output_dict['outputs']

**RUN Locust for end piont can be MME or normal**

In [ ]:
cw_start = datetime.datetime.utcnow()
!/home/ec2-user/anaconda3/envs/python3/bin/locust -f locust_script.py -u 50 --headless --host=http://$endpoint_name_p5 --stop-timeout 90 -L DEBUG -t 5m --logfile=logfile.log --csv=payload_json.json --csv-full-history --reset-stats              
cw_end = datetime.datetime.utcnow()        


**RUN MME Now**

In [ ]:
%%time
import json
import boto3
from botocore.config import Config
import pandas as pd
import itertools
import datetime

region = 'us-east-1'
content_type = 'text/csv'
endpoint_name_p5_mme = 'p5-bert-uc-mme-2022-09-08-19-55-35-829'

boto3config = Config(
    retries={
        'max_attempts': 100,
        'mode': 'standard'
    }
)


sagemaker_client = boto3.client('sagemaker-runtime',
                                     config=boto3config,
                                     region_name=region)

max_seq_length=512
print(f"Leverage the stored:ids: max_seq_length={max_seq_length}:: create above when creating the model : INPUTSid's and attention masks read earlier")
#input_ids, attention_mask = tokenize_text(text_triton, enc, max_length=max_seq_length)

payload = {
    "inputs": [
        {"name": "INPUT__0", "shape": [1, max_seq_length], "datatype": "INT32", "data": input_ids},
        {"name": "INPUT__1", "shape": [1, max_seq_length], "datatype": "INT32", "data": attention_mask},
    ]
}

response = sagemaker_client.invoke_endpoint(
    EndpointName=endpoint_name_p5_mme, ContentType="application/octet-stream", Body=json.dumps(payload), TargetModel  = "/model-9/model.tar.gz"
)

output_dict = json.loads(response["Body"].read().decode("utf8"))

# -- output_dict['outputs'][0]['data']  -- has 0 and 1 as 2 indexes in list 
print(output_dict.keys())


response = sagemaker_client.invoke_endpoint(
    EndpointName=endpoint_name_p5, ContentType="text/json", Body=json.dumps(payload)
)
output_dict = json.loads(response["Body"].read().decode("utf8"))

# -- output_dict['outputs'][0]['data']  -- has 0 and 1 as 2 indexes in list 
print(output_dict.keys())


In [ ]:
output_dict['outputs']

### Previous Runs

In [ ]:
cw_start = datetime.datetime.utcnow()
!/home/ec2-user/anaconda3/envs/python3/bin/locust -f locust_script.py -u 50 --headless --host=http://sagemaker-decision-trees-2021-03-17-03-32-20-604 --stop-timeout 90 -L DEBUG -t 5m --logfile=logfile.log --csv=locust.csv --csv-full-history --reset-stats              
cw_end = datetime.datetime.utcnow()        

In [ ]:
locust_data = pd.read_csv('locust.csv_stats.csv')
for index, row in locust_data.head(n=2).iterrows():
     print(index, row)

In [ ]:
import math
import time

statistics = ['Sum', 'SampleCount', 'Average', 'Minimum', 'Maximum']
extended = ['p50', 'p90', 'p95', 'p99', 'p100']

metrics_to_gather = []
metrics_to_gather.append('CPUUtilization')
metrics_to_gather.append('MemoryUtilization')
metrics_to_gather.append('GPUUtilization')
metrics_to_gather.append('GPUMemoryUtilization')
metrics_to_gather.append('DiskUtilization')
metrics_to_gather.append('ModelLatency')
metrics_to_gather.append('OverheadLatency')
metrics_to_gather.append('Invocations')
metrics_to_gather.append('Invocation4XXErrors')
metrics_to_gather.append('Invocation5XXErrors')
metrics_to_gather.append('InvocationsPerInstance')
#metrics_to_gather.append('ModelLoadingTime')
#metrics_to_gather.append('ModelCacheHit')
#metrics_to_gather.append('ModelDownloadingTime')
#metrics_to_gather.append('ModelLoadingWaitTime')
#metrics_to_gather.append('ModelUnloadingTime')
#metrics_to_gather.append('LoadedModelCount')

def get_sample_count(cw_end, cw_start):
    
    cloudwatch = boto3.client('cloudwatch')
    metrics_to_gather=['Invocations']
    statistics=['SampleCount']
    
    # Period must be 1, 5, 10, 30, or multiple of 60
    # Calculate closest multiple of 60 to the total elapsed time
    factor = math.ceil((cw_end - cw_start).total_seconds() / 60)
    period = factor * 60
    
    cloudwatch_ready = False
    
    # Keep polling CloudWatch metrics until datapoints are available
    while not cloudwatch_ready:
        time.sleep(90)
        for metric in metrics_to_gather:
            model_latency_metrics = cloudwatch.get_metric_statistics(MetricName=metric,
                                             Dimensions=[{'Name': 'EndpointName',
                                                          'Value': endpoint_name_p5},
                                                         {'Name': 'VariantName',
                                                          'Value': "AllTraffic"}],
                                             Namespace="AWS/SageMaker",
                                             StartTime=cw_start,
                                             EndTime=cw_end,
                                             Period=period,
                                             Statistics=statistics
                                             )
            #print(metric)
            if len(model_latency_metrics['Datapoints']) > 0:
                samplecount = model_latency_metrics['Datapoints'][0]['SampleCount']
                cloudwatch_ready = True
    
    return(samplecount)

def collect_cloudwatch_metrics(statistics, extended, metrics_to_gather, total_runs, cw_end, cw_start):
    
    print('Getting Cloudwatch:')
    cloudwatch = boto3.client('cloudwatch')

    # Period must be 1, 5, 10, 30, or multiple of 60
    # Calculate closest multiple of 60 to the total elapsed time
    factor = math.ceil((cw_end - cw_start).total_seconds() / 60)
    period = factor * 60
    print('Time elapsed: {} seconds'.format((cw_end - cw_start).total_seconds()))
    print('Using period of {} seconds\n'.format(period))

    cloudwatch_ready = False
    
    # Keep polling CloudWatch metrics until datapoints are available
    while not cloudwatch_ready:
        
        time.sleep(90)
        
        print('Waiting 30 seconds ...')

        for metric in metrics_to_gather:
            
            if(metric.find('Util') != -1):
                namespace = "/aws/sagemaker/Endpoints"
            else:
                namespace = "AWS/SageMaker"
            
            model_latency_metrics = cloudwatch.get_metric_statistics(MetricName=metric,
                                             Dimensions=[{'Name': 'EndpointName',
                                                          'Value': endpoint_name_p5_mme},
                                                         {'Name': 'VariantName',
                                                          'Value': "AllTraffic"}],
                                             Namespace=namespace,
                                             StartTime=cw_start,
                                             EndTime=cw_end,
                                             Period=period,
                                             Statistics=statistics,
                                             ExtendedStatistics=extended
                                             )            
            #print(metric)
            if len(model_latency_metrics['Datapoints']) > 0:
                #print(model_latency_metrics)
                print(metric +'\n')
                side_avg = model_latency_metrics['Datapoints'][0]['Average'] / total_runs
                side_p50 = model_latency_metrics['Datapoints'][0]['ExtendedStatistics']['p50'] / total_runs
                side_p90 = model_latency_metrics['Datapoints'][0]['ExtendedStatistics']['p90'] / total_runs
                side_p95 = model_latency_metrics['Datapoints'][0]['ExtendedStatistics']['p95'] / total_runs
                side_p99 = model_latency_metrics['Datapoints'][0]['ExtendedStatistics']['p99'] / total_runs
                side_p100 = model_latency_metrics['Datapoints'][0]['ExtendedStatistics']['p100'] / total_runs

                sumcount = model_latency_metrics['Datapoints'][0]['Sum']
                samplecount = model_latency_metrics['Datapoints'][0]['SampleCount']
                average = model_latency_metrics['Datapoints'][0]['Average']
                minimum = model_latency_metrics['Datapoints'][0]['Minimum']
                maximum = model_latency_metrics['Datapoints'][0]['Maximum']

                #statistics = ['Sum', 'SampleCount', 'Average', 'Minimum', 'Maximum']
                #extended = ['p50', 'p90', 'p95', 'p99', 'p100']
                print('Avg | P50 | P90 | P95 | P95 | P100')
                print('{:.4f} | {:.4f} | {:.4f} | {:.4f} | {:.4f}\n'.format(side_avg, side_p50, side_p90, side_p95, side_p99, side_p100))
                print('Sum | SampleCount | Average | Minimum | Maximum')
                print('{:.4f} | {:.4f} | {:.4f} | {:.4f} | {:.4f}\n'.format(sumcount, samplecount, average, minimum, maximum))
                
                cloudwatch_ready = True


In [ ]:
total_runs = get_sample_count(cw_end, cw_start)
collect_cloudwatch_metrics(statistics, extended, metrics_to_gather, total_runs, cw_end, cw_start)